In [1]:
import torch
import transformers

from datasets import load_dataset
import numpy as np

from project import CreditDefault, PII, SkinCancer, set_seed

set_seed(1234)

## Credit-card default

In [3]:
credit = CreditDefault()
credit.prepare_dataset("cuda")

In [6]:
optimizers = {
    "adamw": (torch.optim.AdamW, {"lr": 2e-5}),
    "rmsprop": (torch.optim.RMSprop, {"lr": 2e-5}),
    "adagrad": (torch.optim.Adagrad, {"lr": 2e-5}),
}

schedulers = {
    "linear": (torch.optim.lr_scheduler.LinearLR, {"start_factor": 0.01, "end_factor": 0.0001, "total_iters": 2}),
    "cosine": (torch.optim.lr_scheduler.CosineAnnealing, {"T_max": 100, "eta_min": 0.01}),
    "polynomial": (torch.optim.lr_scheduler.PolynomialLR, {"total_iters": 100, "power": 2})
}

In [7]:
df = credit.train(
    result_csv = "test.csv", 
    optimizers = optimizers,
    schedulers = schedulers,
    epochs = 5,
    batch_size = 1000,
    device = "cuda",
    loss_func = torch.nn.MSELoss
    )

-------------------------------------
|linear, adamw|
Epoch 1
	batch 1 | loss: 0.4424096345901489 | accuracy: 0.537
	batch 2 | loss: 0.4221212863922119 | accuracy: 0.556
	batch 3 | loss: 0.41576284170150757 | accuracy: 0.563
	batch 4 | loss: 0.3999929428100586 | accuracy: 0.584
	batch 5 | loss: 0.3750000596046448 | accuracy: 0.605
	batch 6 | loss: 0.35946759581565857 | accuracy: 0.618
	batch 7 | loss: 0.3029974699020386 | accuracy: 0.673
	batch 8 | loss: 0.3039945960044861 | accuracy: 0.68
	batch 9 | loss: 0.24527829885482788 | accuracy: 0.725
	batch 10 | loss: 0.24620354175567627 | accuracy: 0.715
	batch 11 | loss: 0.2339823693037033 | accuracy: 0.742
	batch 12 | loss: 0.23700402677059174 | accuracy: 0.746
	batch 13 | loss: 0.22199484705924988 | accuracy: 0.767
	batch 14 | loss: 0.2247716784477234 | accuracy: 0.767
	batch 15 | loss: 0.23900000751018524 | accuracy: 0.755
	batch 16 | loss: 0.2389996200799942 | accuracy: 0.753
	batch 17 | loss: 0.2141554355621338 | accuracy: 0.78
	batch 

## PII

In [2]:
model_path = "distilbert/distilroberta-base"

pii = PII()
pii.load_tokenizer(model_path)
pii.prepare_dataset()

In [7]:
optimizers = {
    "adamw": (torch.optim.AdamW, {"lr": 2e-5}),
    "rmsprop": (torch.optim.RMSprop, {"lr": 2e-5, "momentum": 0}),
    "adagrad": (torch.optim.Adagrad, {"lr": 3e-5}),
}

schedulers = {
    "linear": {"num_warmup_steps": 0},
    "cosine": {"num_warmup_steps": 0},
    "polynomial": {"num_warmup_steps": 0, "power": 2}
}

In [8]:
df = pii.train(
    output_dir = "test/",
    optimizers = optimizers,
    schedulers = schedulers,
    tokenizer_path = model_path,
    model_path = model_path,
    strategy = "epoch",
    epochs = 10,
    batch_size = 150,
    result_csv = "test.csv",
    device = "cuda"
)

----------------------------
Optimizer: adamw | Scheduler: linear


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.13120447099208832, 'eval_precision': 0.7934772788869816, 'eval_recall': 0.8295239894219871, 'eval_f1': 0.8111003370734635, 'eval_accuracy': 0.9619373446657052, 'eval_runtime': 3.9377, 'eval_samples_per_second': 825.357, 'eval_steps_per_second': 5.587, 'epoch': 1.0}
{'eval_loss': 0.06966577470302582, 'eval_precision': 0.9029126213592233, 'eval_recall': 0.9222704948998867, 'eval_f1': 0.9124889034247536, 'eval_accuracy': 0.980040501979074, 'eval_runtime': 4.2822, 'eval_samples_per_second': 758.952, 'eval_steps_per_second': 5.138, 'epoch': 2.0}
{'eval_loss': 0.05863739550113678, 'eval_precision': 0.9199440820130476, 'eval_recall': 0.9322818284850775, 'eval_f1': 0.9260718641523594, 'eval_accuracy': 0.9825565340124575, 'eval_runtime': 4.0709, 'eval_samples_per_second': 798.356, 'eval_steps_per_second': 5.404, 'epoch': 3.0}
{'eval_loss': 0.053025200963020325, 'eval_precision': 0.9270378287944976, 'eval_recall': 0.9420098224404987, 'eval_f1': 0.9344638590902703, 'eval_accuracy'

Checkpoint destination directory test//adamw_linear\checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 0.1533, 'grad_norm': 0.6746379733085632, 'learning_rate': 9.316239316239318e-06, 'epoch': 5.32}
{'eval_loss': 0.05085839331150055, 'eval_precision': 0.9328066914498141, 'eval_recall': 0.9479599546656592, 'eval_f1': 0.940322278433577, 'eval_accuracy': 0.9856402074192262, 'eval_runtime': 3.9764, 'eval_samples_per_second': 817.332, 'eval_steps_per_second': 5.533, 'epoch': 6.0}
{'eval_loss': 0.05173797905445099, 'eval_precision': 0.9363069187075147, 'eval_recall': 0.9496599924442766, 'eval_f1': 0.9429361841796784, 'eval_accuracy': 0.9862078487926115, 'eval_runtime': 3.98, 'eval_samples_per_second': 816.583, 'eval_steps_per_second': 5.528, 'epoch': 7.0}
{'eval_loss': 0.05042613670229912, 'eval_precision': 0.9377736376339078, 'eval_recall': 0.9507933509633547, 'eval_f1': 0.9442386155794213, 'eval_accuracy': 0.9864686569911939, 'eval_runtime': 4.1097, 'eval_samples_per_second': 790.819, 'eval_steps_per_second': 5.353, 'epoch': 8.0}
{'eval_loss': 0.05087592825293541, 'eval_precision':

...

{'loss': 0.2831, 'grad_norm': 1.14810049533844, 'learning_rate': 1.3392022563188283e-05, 'epoch': 5.32}
{'eval_loss': 0.13840194046497345, 'eval_precision': 0.7850366923214606, 'eval_recall': 0.8284850774461655, 'eval_f1': 0.8061759029500966, 'eval_accuracy': 0.9605872786965727, 'eval_runtime': 3.9551, 'eval_samples_per_second': 821.718, 'eval_steps_per_second': 5.562, 'epoch': 6.0}
{'eval_loss': 0.13562233746051788, 'eval_precision': 0.7876351290985437, 'eval_recall': 0.8326407253494522, 'eval_f1': 0.8095128781965933, 'eval_accuracy': 0.9614003866098002, 'eval_runtime': 4.1635, 'eval_samples_per_second': 780.601, 'eval_steps_per_second': 5.284, 'epoch': 7.0}
{'eval_loss': 0.13425825536251068, 'eval_precision': 0.7899919434249396, 'eval_recall': 0.8334907442387609, 'eval_f1': 0.8111586010386507, 'eval_accuracy': 0.961691878125863, 'eval_runtime': 4.0175, 'eval_samples_per_second': 808.965, 'eval_steps_per_second': 5.476, 'epoch': 8.0}
{'eval_loss': 0.13379499316215515, 'eval_precision'

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.29934048652648926, 'eval_precision': 0.5962062000526916, 'eval_recall': 0.6411975821684927, 'eval_f1': 0.6178839590443685, 'eval_accuracy': 0.9192875333681078, 'eval_runtime': 3.9493, 'eval_samples_per_second': 822.921, 'eval_steps_per_second': 5.571, 'epoch': 1.0}
{'eval_loss': 0.22242829203605652, 'eval_precision': 0.6837509822753863, 'eval_recall': 0.7396108802417831, 'eval_f1': 0.7105848192005807, 'eval_accuracy': 0.9389401982142309, 'eval_runtime': 3.9487, 'eval_samples_per_second': 823.047, 'eval_steps_per_second': 5.571, 'epoch': 2.0}
{'eval_loss': 0.1935511827468872, 'eval_precision': 0.7123072903057225, 'eval_recall': 0.7723838307517945, 'eval_f1': 0.7411300919842312, 'eval_accuracy': 0.9456751864011537, 'eval_runtime': 3.9712, 'eval_samples_per_second': 818.384, 'eval_steps_per_second': 5.54, 'epoch': 3.0}
{'eval_loss': 0.18016143143177032, 'eval_precision': 0.7315641968821159, 'eval_recall': 0.7889119758216849, 'eval_f1': 0.7591565936562755, 'eval_accuracy': 

Checkpoint destination directory test//adagrad_polynomial\checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 0.3092, 'grad_norm': 1.234157919883728, 'learning_rate': 6.58772554605888e-06, 'epoch': 5.32}
{'eval_loss': 0.1688489019870758, 'eval_precision': 0.7424349049964813, 'eval_recall': 0.7971288250850019, 'eval_f1': 0.7688103479686645, 'eval_accuracy': 0.952041974778313, 'eval_runtime': 3.9689, 'eval_samples_per_second': 818.868, 'eval_steps_per_second': 5.543, 'epoch': 6.0}
{'eval_loss': 0.1669256091117859, 'eval_precision': 0.7442126573365021, 'eval_recall': 0.7985455232338496, 'eval_f1': 0.77042234270354, 'eval_accuracy': 0.9525329078579976, 'eval_runtime': 4.0167, 'eval_samples_per_second': 809.13, 'eval_steps_per_second': 5.477, 'epoch': 7.0}
{'eval_loss': 0.16586633026599884, 'eval_precision': 0.746079295154185, 'eval_recall': 0.7997733282961843, 'eval_f1': 0.7719938007110949, 'eval_accuracy': 0.9528243993740603, 'eval_runtime': 4.0116, 'eval_samples_per_second': 810.144, 'eval_steps_per_second': 5.484, 'epoch': 8.0}
{'eval_loss': 0.16548708081245422, 'eval_precision': 0.746

## Melanoma identification

In [2]:
skincancer_model_path = "apple/mobilevit-small"

skin_cancer = SkinCancer()
skin_cancer.load_image_processor(skincancer_model_path)
skin_cancer.prepare_dataset()

In [3]:
optimizers = {
    "adamw": (torch.optim.AdamW, {"lr": 2e-4}),
    "rmsprop": (torch.optim.RMSprop, {"lr": 2e-4, "momentum": 0}),
    "adagrad": (torch.optim.Adagrad, {"lr": 3e-5}),
}

schedulers = {
    "linear": {"num_warmup_steps": 0},
    "cosine": {"num_warmup_steps": 0},
    "polynomial": {"num_warmup_steps": 0, "power": 2}
}

In [4]:
df = skin_cancer.train(
    output_dir = "test/",
    result_csv = "test.csv",
    optimizers = optimizers,
    schedulers = schedulers,
    image_processor_path= skincancer_model_path,
    model_path = skincancer_model_path,
    epochs = 1,
    batch_size = 24,
    strategy="epoch",
    device = "cuda"
)

----------------------------
Optimizer: adamw | Scheduler: linear


Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 640]) in the checkpoint and torch.Size([7, 640]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: MobileViTForImageClassification.forward() got an unexpected keyword argument 'age'

In [41]:
#dataset = load_dataset(path = "ai4privacy/pii-masking-200k", data_files = "english_pii_43k.jsonl")
dataset = load_dataset(path = "conll2003")

In [5]:
dataset = load_dataset("marmal88/skin_cancer")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'image_id', 'lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization'],
        num_rows: 9577
    })
    validation: Dataset({
        features: ['image', 'image_id', 'lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization'],
        num_rows: 2492
    })
    test: Dataset({
        features: ['image', 'image_id', 'lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization'],
        num_rows: 1285
    })
})

In [14]:
dataset["train"][0]["dx"]

'actinic_keratoses'

SGD Algorithms: 
- "adamw_torch"
- "adagrad"
- "rmsprop"


LR Schedulers:

- "cosine"
- "inverse_sqrt"

inverse_sqrt = lambda step: 1/math.sqrt(step, 100)

scheduler = LambdaLR(optimizer, lr_lambda = inverse_sqrt)


- "reduce_lr_on_plateau"